### Init

In [272]:
import robotic as ry
import time
with open("logger.txt","w") as f:
    f.write("")
try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing_bare.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.view()

0

### Komo probs

In [273]:
way0 = C.addFrame('way0', 'start_area')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0.2 0.15 0.1) d(90 0 0 0)')

way1 = C.addFrame('way1', 'start_area')

way1.setShape(ry.ST.marker, size=[.2])
way1.setPosition(C.getFrame('l_gripper').getPosition())
#C.addFrame("marker22","cargo").setShape(ry.ST.marker,[.3]).setPosition([0,0,0])
C.view()

0

In [274]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [275]:
def create_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [276]:

def throw_position_komo(C, gripper_name, grasp_frame_name):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way0'], ry.OT.eq, [1e1], [0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret2)
    return komo

In [277]:
def post_throw_position_komo(C, gripper_name, throw_waypoint_frame):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, throw_waypoint_frame], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way1'], ry.OT.eq, [1e1], [0])
    ret3 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret3)
    return komo

In [278]:
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path0 = komo0.getPath()

komo = create_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()

komo2 = throw_position_komo(C, 'l_gripper', 'cargo')
path2 = komo2.getPath()

komo3 = post_throw_position_komo(C, 'l_gripper', 'way1')
path3 = komo3.getPath()

{ time: 0.004088, evals: 26, done: 1, feasible: 1, sos: 0.20516, f: 0, ineq: 0, eq: 0.00545494 }
{ time: 0.003552, evals: 26, done: 1, feasible: 1, sos: 0.205347, f: 0, ineq: 0, eq: 0.00423111 }
{ time: 0.000529, evals: 9, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00322506 }
{ time: 0.000332, evals: 4, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0169886 }


### Pick up ball

In [279]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperCloseGrasp(ry._left, 'cargo',speed=1000000000000,width=0.06) 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

#C.getFrame('cargo').unLink()
C.attach('l_gripper', 'cargo')

# print(path2.shape)
# bot.move(path2, [3.])
# while bot.getTimeToEnd() > 0:
#     bot.sync(C, .1)

# pre_throw_pos = C.getFrame('l_gripper').getPosition()

# bot.move(path3, [3.])
# bot.gripperMove(ry._left, width=0.075, speed=5)
# while bot.getTimeToEnd() > 0:
#     bot.sync(C, .1)


# bot.home(C)

### Calculate throw

In [280]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from my_utils import rotation_matrix_to_quaternion, velocity_to_rotation_matrix, find_line_intersections_2d
def calculate_throw_parameters_bin(initial_position, bin_dimensions, gravity=9.81):
    bin_pos = C.getFrame("bin").getPosition()
    target_vector = np.array([bin_pos[0] - initial_position[0], bin_pos[1] - initial_position[1], 0])
    target_vector_length = np.linalg.norm(target_vector)
    target_vector /= target_vector_length  # Normalize the vector
    x = np.abs(bin_pos[0]-initial_position[0])
    y = np.abs(bin_pos[1]-initial_position[1])
    x_y_dist = np.sqrt(np.square(x) + np.square(y))
    init_pos_2d = np.array([initial_position[0],initial_position[1]])
    intersections = find_line_intersections_2d(bin_pos, initial_position, bin_dimensions)
    print(intersections)
    min_projection = np.min([np.linalg.norm(intersection - init_pos_2d) for intersection in intersections])
    max_projection = np.max([np.linalg.norm(intersection - init_pos_2d) for intersection in intersections])
    def loss_function(params):
        release_height, v_magnitude, theta_xz = params
        x0, y0, z0 = initial_position
        bottom_left = bin_dimensions['bottom_left']
        top_right = bin_dimensions['top_right']
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        min_z_target = bottom_left[2]
        max_z_target = top_right[2]
        a = 0.5 * gravity
        b = -v_z
        c_min = release_height - min_z_target
        c_max = release_height - max_z_target
        b_square = np.square(b)
        discriminant_min = b_square - 4 * a * c_min
        discriminant_max = b_square - 4 * a * c_max
        # Check if the trajectory is valid for both boundaries
        if discriminant_min < 0 and discriminant_max < 0:
            return float("inf")  # No valid trajectory

        # Calculate the times for both height boundaries
        t_flight_min = (-b + np.sqrt(discriminant_min)) / (2 * a) if discriminant_min >= 0 else float("inf")
        t_flight_max = (-b + np.sqrt(discriminant_max)) / (2 * a) if discriminant_max >= 0 else float("inf")
        # Choose the shortest valid time
        t_flight = min(t_flight_min, t_flight_max)
        
        x_landing = v_x*t_flight
        withing_projection_bounds = min_projection<= x_landing <= max_projection
        if withing_projection_bounds:
            with open("logger.txt","a") as f:
                f.write(f"Insideee!!!!!Release height: {release_height} Velx: {v_x} loss:0 x_landing:{x_landing} min/max:{min_projection}/{max_projection}\n")
            return 0 
        else:
            dproj = max(0,x_landing-min_projection, max_projection-x_landing)
            with open("logger.txt","a") as f:
                f.write(f"Release height: {release_height} Velx: {v_x} loss:{dproj} x_landing:{x_landing} min/max:{min_projection}/{max_projection}\n")
        #dx = np.abs(x_landing-x_y_dist)
        return dproj 


    initial_guess = [initial_position[2], 4.0, np.pi / 4]
    
    bounds = [(initial_position[2]+0.2, 1.5),
                (0.5, 10.0), 
                (0, np.pi / 3)] 
    
    #result = minimize(loss_function, initial_guess, bounds=bounds, method='SLSQP')
    result = differential_evolution(loss_function, bounds)
    if result.success:
        release_height, v_magnitude, theta_xz = result.x
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        bottom_left = bin_dimensions['bottom_left']
        top_right = bin_dimensions['top_right']
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        min_z_target = bottom_left[2]
        max_z_target = top_right[2]
        a = 0.5 * gravity
        b = -v_z
        c_min = release_height - min_z_target
        c_max = release_height - max_z_target
        b_square = np.square(b)
        discriminant_min = b_square - 4 * a * c_min
        discriminant_max = b_square - 4 * a * c_max
        # Check if the trajectory is valid for both boundaries
        if discriminant_min < 0 and discriminant_max < 0:
            return float("inf")  # No valid trajectory

        # Calculate the times for both height boundaries
        t_flight_min = (-b + np.sqrt(discriminant_min)) / (2 * a) if discriminant_min >= 0 else float("inf")
        t_flight_max = (-b + np.sqrt(discriminant_max)) / (2 * a) if discriminant_max >= 0 else float("inf")
        # Choose the shortest valid time
        t_flight = min(t_flight_min, t_flight_max)
        
        x_landing = v_x*t_flight
        withing_projection_bounds = min_projection<= x_landing <= max_projection
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Finally found: inside? {withing_projection_bounds}")
        print(f"is positive x: {((bin_pos[0]-initial_position[0])/np.abs(bin_pos[0]-initial_position[0]))}")
        print(f"initial v_x = {v_x}")
        v_y = v_x* (y/x_y_dist) * ((bin_pos[1]-initial_position[1])/np.abs(bin_pos[1]-initial_position[1]))
        v_x = v_x* (x/x_y_dist) * ((bin_pos[0]-initial_position[0])/np.abs(bin_pos[0]-initial_position[0]))
        print(f"After v_x:{v_x} and v_y:{v_y}")
        return {
            'release_position': [initial_position[0], initial_position[1], release_height],
            'velocity': [v_x, v_y, v_z],
            'angle_xz': theta_xz,
        }
    else:
       raise ValueError("Optimization failed to find valid parameters.")


initial_pos = C.getFrame("cargo").getPosition()


bin_dims = {
   'bottom_left': (1.75, 1.75, 0.08),
   'top_right': (2.25, 2.25, 0.15) 
}
C.addFrame("bottom_left").setShape(ry.ST.marker,[.2]).setPosition([1.75, 1.75, 0.1]).setColor([1,1,0])
C.addFrame("top_right").setShape(ry.ST.marker,[.2]).setPosition([2.25, 2.25, 0.2]).setColor([1,1,0])
C.view()
result = calculate_throw_parameters_bin(initial_pos, bin_dims)


print("Release Position:", result['release_position'])
print("Velocity:", result['velocity'])
print("Release Angle (radians):", result['angle_xz'])




C.addFrame("release_frame").setPosition(result['release_position']).setShape(ry.ST.marker,[.4]).setColor([1,0,0]).setContact(0).setQuaternion(rotation_matrix_to_quaternion(velocity_to_rotation_matrix(result["velocity"])))


C.view()
#finding the initial point from the release point and the velocity
#find the backward kinematics and simulate it


def find_initial_point_from_release(release_position, release_velocity, robot_base, height_threshold=0.25, distance_threshold=1.53):
   """
   Finds the initial position for the throwing motion based on the release position and velocity.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.
       height_threshold: Minimum allowable height for the initial position.
       distance_threshold: Maximum allowable distance from the robot base.


   Returns:
       initial_position: The computed initial position, if found.
   """
   # Compute the initial position using backward kinematics
   time_step = 0.001  # Small time step for simulation
   gravity = np.array([0, 0, -9.81])  # Gravity vector


   # Backward integrate to find the initial position
   velocity = np.array(release_velocity)
   position = np.array(release_position)
   distance_to_base = np.linalg.norm(position - np.array(robot_base))
   iteration = 0
   prev_pos = position
   while position[2] > height_threshold and distance_to_base <= distance_threshold:  # Ensure height is above threshold
       iteration+=1
       prev_pos = position
       position -= velocity * time_step
       velocity -= gravity * time_step
       # Check distance to the robot base
       distance_to_base = np.linalg.norm(position - np.array(robot_base))
       #print(f"Iter {iteration}")
   iteration = 0
   velocity = np.array(release_velocity)
   time_step = .1
   position = np.array(release_position)


   while position[2] > height_threshold:
       position += velocity * time_step
       velocity += gravity * time_step
       iteration += 1
       frame = C.addFrame(f"trajectory-{iteration}").setShape(ry.ST.marker,[.2]).setColor([0,0,1]).setPosition(position)
       C.view()
       print(f"Frame {iteration} added with position {frame.getPosition()} new vel:{velocity}")
   return prev_pos  # No valid initial position found






def pick_last_object_if_valid(release_position, release_velocity, robot_base):
   """
   Simulates the robot to pick the last object if conditions are met.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.


   Returns:
       None
   """
   # Find the initial position
   initial_position = find_initial_point_from_release(release_position, release_velocity, robot_base)


   if initial_position is not None:
       print(f"Initial Position Found: {initial_position}")
       # Visualize the initial position
       C.addFrame("initial_position").setPosition(initial_position).setShape(ry.ST.marker, [0.4]).setContact(0).setColor([0,1,0]).setQuaternion(rotation_matrix_to_quaternion(velocity_to_rotation_matrix(release_velocity)))
       C.view()
       return initial_position
       # Simulate the robot to pick the last object
       # Add your grasping or motion planning logic here
   else:
       print("No valid initial position found.")


initial_pos = pick_last_object_if_valid(np.array(result['release_position']),np.array(result["velocity"]),C.getFrame("l_panda_base").getPosition())


#go to inital position with y axis being scalar to the velocity vector
def goto_init(gripper_name, initial_pos_name,release_frame_name):
   q0=C.getJointState()
   komo = ry.KOMO(C, 1, 1, 0, True)
   komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [0, 0, 0],order=0)
   komo.addObjective([1], ry.FS.scalarProductXZ, [initial_pos_name, gripper_name], ry.OT.eq, [1e1], [-1],order=0)
   komo.addObjective([1], ry.FS.scalarProductYX, [initial_pos_name, gripper_name], ry.OT.eq, [1e1], [1],order=0)
   ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
   print(ret2)
   return komo


def throw_komo(gripper_name,release_frame_name,velocity):
   q0 = C.getJointState()
   v_magnitude = np.linalg.norm(np.array(velocity))
   komo = ry.KOMO(C, 1, 1, 1, True)
   komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, release_frame_name], ry.OT.eq, [1e1], [0, 0, 0],order=0)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, release_frame_name], ry.OT.sos, [1e-2])
   #komo.addObjective([], ry.FS.scalarProductXZ, [release_frame_name,gripper_name], ry.OT.eq, [1e1], [-1],order=0)
   #komo.addObjective([1], ry.FS.scalarProductYX, [release_frame_name, gripper_name], ry.OT.eq, [1e1], [1],order=0)
   komo.addObjective([1.], ry.FS.position, [gripper_name], ry.OT.eq, [1e1], velocity,order=1)
   #print(C.eval(ry.FS.positionDiff, ['l_gripper',"release_frame"], order=1))
   ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
   print(komo.report())
   print(ret2)
   return komo




[[1.75, np.float64(2.000091537220649)], [2.25, np.float64(1.9999084627793506)]]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Finally found: inside? True
is positive x: 1.0
initial v_x = 2.496644457358949
After v_x:2.496644290002641 and v_y:-0.0009141435170269341
Release Position: [np.float64(0.6495107412338257), np.float64(2.0004944801330566), np.float64(0.30680511796257937)]
Velocity: [np.float64(2.496644290002641), np.float64(-0.0009141435170269341), np.float64(2.8967490380708973)]
Release Angle (radians): 0.8594467249420338
Frame 1 added with position [0.89917517 2.00040307 0.59648002] new vel:[ 2.49664429e+00 -9.14143517e-04  1.91574904e+00]
Frame 2 added with position [1.1488396  2.00031165 0.78805493] new vel:[ 2.49664429e+00 -9.14143517e-04  9.34749038e-01]
Frame 3 added with position [1.39850403 2.00022024 0.88152983] new vel:[ 2.49664429e+00 -9.14143517e-04 -4.62509619e-02]
Frame 4 added with position [1.64816846 2.00012882 0.87690473] new vel:[ 2.49664429e+00 -9.14143517e-04 -1.02725096

In [ ]:
#throwing ops
komo4 = goto_init('l_gripper','initial_position',"release_frame")
path4 = komo4.getPath()
komo5 = throw_komo("l_gripper","release_frame",result["velocity"])
path5 = komo5.getPath()
bot.move(path4, [3.])
bot.move(path5, [3.])
print(f"bot op time: {bot.getTimeToEnd()}")
isApplied = False
while bot.getTimeToEnd() > 0:
    if bot.getTimeToEnd()<=0.07 and not isApplied:
        print(f"Opening gripper")
        bot.gripperMove(ry._left,speed=10,width=0.08) 
        isApplied = True 
    bot.sync(C, .1)

{ time: 0.003715, evals: 64, done: 1, feasible: 1, sos: 0.0832922, f: 0, ineq: 0, eq: 0.000592885 }
{'o2': {'name': 'F_PositionDiff', 'frames': ['l_gripper', 'release_frame'], 'err': 3.764179281264423e-05, 'order': 0.0, 'type': 'sos', 'scale': [0.01], 'times': [1.0], 'slices': [0.0, 0.0]}, 'o0': {'name': 'F_qItself', 'frames': ['#14'], 'err': 0.019551880192325326, 'order': 0.0, 'type': 'sos', 'scale': [0.1], 'target': [0.19167287647724057, 0.8061635494232177, -0.20014630258083263, -1.763914704322815, 0.2812256515026095, 2.5419363975524902, 0.6005197167396547], 'times': [], 'slices': [0.0, 0.0]}, 'o1': {'name': 'F_PositionDiff', 'frames': ['l_gripper', 'release_frame'], 'err': 7.395244831310064, 'order': 0.0, 'type': 'eq', 'scale': [10.0], 'target': [0.0, 0.0, 0.0], 'times': [1.0], 'slices': [0.0, 0.0]}, 'o3': {'name': 'F_Position', 'frames': ['l_gripper'], 'err': 44.08967071518046, 'order': 1.0, 'type': 'eq', 'scale': [10.0], 'target': [2.496644290002641, -0.0009141435170269341, 2.8967

In [282]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[0])
release_frame.setPosition(result['release_position'])
C.view()
time.sleep(15)

### Clean up

In [283]:
del C
del bot